### Import Necessary Packages

In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import fuzzymatcher

In [4]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzymatcher import link_table, fuzzy_left_join

### Load & Clean CSV Files

In [9]:
# descriptors of cannabis strains
can = pd.read_csv('/Users/stephaniekendall/Desktop/Flatiron/projects/Cannabis-Predictions/CSV Files/cannabis.csv', error_bad_lines=False)
#2k+ datapoints with type, effects, rating, and taste

# kushy csv
strains = pd.read_csv('/Users/stephaniekendall/Desktop/Flatiron/projects/Cannabis-Predictions/CSV Files/strains-kushy_api.2017-11-14.csv')
#1k+ datapoints with type, effects, thc, cbd (not beautiful)

b'Skipping line 1333: expected 6 fields, saw 10\nSkipping line 1655: expected 6 fields, saw 10\nSkipping line 1656: expected 6 fields, saw 10\nSkipping line 1657: expected 6 fields, saw 10\nSkipping line 2102: expected 6 fields, saw 10\n'


In [99]:
can.head()

,strain,type,rating,effects,taste,description
0,100-og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-white-widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24k-gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [100]:
can = can.dropna()
can = can.drop(columns=['description'])
can.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2273 entries, 0 to 2347
Data columns (total 5 columns):
strain     2273 non-null object
type       2273 non-null object
rating     2273 non-null object
effects    2273 non-null object
taste      2273 non-null object
dtypes: object(5)
memory usage: 106.5+ KB


In [6]:
strains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 31 columns):
id             9524 non-null int64
status         9524 non-null int64
sort           9524 non-null int64
name           9523 non-null object
slug           0 non-null float64
image          0 non-null float64
description    1 non-null object
type           9500 non-null object
crosses        564 non-null object
breeder        8979 non-null object
effects        1015 non-null object
ailment        971 non-null object
flavor         971 non-null object
location       621 non-null object
terpenes       1 non-null object
thc            9524 non-null int64
thca           8635 non-null float64
thcv           8635 non-null float64
cbd            9524 non-null int64
cbda           8635 non-null float64
cbdv           8635 non-null float64
cbn            9524 non-null int64
cbg            8635 non-null float64
cbgm           8635 non-null float64
cbgv           8635 non-null float64
cbc   

In [11]:
# drop columns with only zeros or NaN
strains = strains.drop(columns=['slug','image','thca','thcv','cbda','cbdv'])

# drop columns with over 80% NaNs
strains = strains.drop(columns=['description','crosses','ailment','flavor','location','terpenes','breeder'])

# drop last 7 columns of empty cbd values
strains = strains.iloc[:, 0:8]

# replace - with whitespace in name 
strains['name'] = strains['name'].str.replace('-',' ')

strains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 8 columns):
id         9524 non-null int64
status     9524 non-null int64
sort       9524 non-null int64
name       9523 non-null object
type       9500 non-null object
effects    1015 non-null object
thc        9524 non-null int64
cbd        9524 non-null int64
dtypes: int64(5), object(3)
memory usage: 595.3+ KB


In [13]:
strains = strains.drop(columns=['sort'])

In [16]:
strains = strains.dropna()
strains.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1015 entries, 0 to 2177
Data columns (total 7 columns):
id         1015 non-null int64
status     1015 non-null int64
name       1015 non-null object
type       1015 non-null object
effects    1015 non-null object
thc        1015 non-null int64
cbd        1015 non-null int64
dtypes: int64(4), object(3)
memory usage: 63.4+ KB


In [18]:
strains = strains.drop(columns=['id','status'])

In [19]:
strains.head()

,name,type,effects,thc,cbd
0,100 OG,Hybrid,Focused,127,16
2,A 10,Indica,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D...",0,0
3,Acapulco Gold,Sativa,"Happy, Euphoric, Uplifted, Relaxed, Creative, ...",0,0
4,Afghani Bullrider,Hybrid,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,...",127,0
5,Afghan Big Bud,Indica,"Euphoric, Happy, Talkative, Relaxed, Sleepy, D...",0,0


In [22]:
can.dropna()
can.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Data columns (total 6 columns):
strain         2347 non-null object
type           2347 non-null object
rating         2347 non-null object
effects        2347 non-null object
taste          2301 non-null object
description    2314 non-null object
dtypes: object(6)
memory usage: 110.1+ KB


In [23]:
can['strain'] = can['strain'].str.title()
can.head()

,strain,type,rating,effects,taste,description
0,100-Og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [24]:
can['strain'] = can['strain'].str.replace('-',' ')

In [31]:
can.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Data columns (total 6 columns):
strain         2347 non-null object
type           2347 non-null object
rating         2347 non-null object
effects        2347 non-null object
taste          2301 non-null object
description    2314 non-null object
dtypes: object(6)
memory usage: 110.1+ KB


In [32]:
strains.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1015 entries, 0 to 2177
Data columns (total 5 columns):
name       1015 non-null object
type       1015 non-null object
effects    1015 non-null object
thc        1015 non-null int64
cbd        1015 non-null int64
dtypes: int64(2), object(3)
memory usage: 47.6+ KB


In [67]:
can['strain'] = can.strain.str.lower()
strains['name'] = strains.name.str.lower()
strains['strain'] = strains['name']

In [50]:
strains.head()

,name,type,effects,thc,cbd,strains
0,100 og,Hybrid,Focused,127,16,100 og
2,a 10,Indica,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D...",0,0,a 10
3,acapulco gold,Sativa,"Happy, Euphoric, Uplifted, Relaxed, Creative, ...",0,0,acapulco gold
4,afghani bullrider,Hybrid,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,...",127,0,afghani bullrider
5,afghan big bud,Indica,"Euphoric, Happy, Talkative, Relaxed, Sleepy, D...",0,0,afghan big bud


In [65]:
type(can.strain[1])

str

In [69]:
strains.strain

0                     100 og
2                       a 10
3              acapulco gold
4          afghani bullrider
5             afghan big bud
8                    afghani
10                  afghooey
11                     afgoo
12                   afgooey
17      alaskan thunder fuck
19                     aloha
23              amnesia haze
24                anesthesia
26              arabian gold
34            bc sweet tooth
35                  berkeley
36               berry white
37                   big bud
39                 big wreck
42              black domina
44          black label kush
47      blackberry hashplant
48           blackberry kush
50                blue bayou
52               blue cheese
58               blue mystic
61            blue satellite
64                 blueberry
68            blueberry kush
70           brainstorm haze
                ...         
2146               tuna kush
2147                   tyson
2148                 u2 kush
2149          

In [36]:
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [56]:
# List for dicts for easy dataframe creation
dict_list = []
# iterating over df with more strains
for name in can.strain:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, strains.strains, 90)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"strain" : name})
    dict_.update({"name" : match[0]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

TypeError: object of type 'float' has no len()

In [ ]:
merge_table.sample(20)

In [158]:
df = pd.merge(merge_table, can, left_on='strain', right_on='strain')
df.head()

,name,strain,type,rating,effects,taste
0,,100 Og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus"
1,,98 White Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel"
2,1024,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody"
3,,13 Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit"
4,,24K Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange"


In [181]:
len(df)

2275

In [159]:
df = df.drop(columns=['name'])
df.sample(20)

,strain,type,rating,effects,taste
1489,Orange Creamsicle,hybrid,4.8,"Relaxed,Happy,Euphoric,Uplifted,Focused","Citrus,Orange,Sweet"
1956,Sublime,sativa,4.2,"Uplifted,Happy,Focused,Euphoric,Talkative","Flowery,Pine,Earthy"
1016,Himalayan Blackberry,indica,4.9,"Sleepy,Relaxed,Euphoric,Happy,Uplifted","Earthy,Berry,Pine"
547,Citrus Punch,sativa,4.2,"Uplifted,Happy,Relaxed,Euphoric,Giggly","Citrus,Orange,Tropical"
909,Grape Ox,indica,4.6,"Sleepy,Relaxed,Hungry,Tingly,Uplifted","Grape,Cheese,Sweet"
1540,Phatt Frutty,indica,4.5,"Uplifted,Happy,Relaxed,Talkative,Creative","Lemon,Spicy/Herbal,Lime"
1205,Lamb S Bread,sativa,4.4,"Euphoric,Creative,Happy,Uplifted,Energetic","Earthy,Woody,Pungent"
851,Glue Tech,hybrid,4.6,"Happy,Relaxed,Hungry,Uplifted,Euphoric","Earthy,Sweet,Berry"
2017,Sweet Hawaiian Tangie Banana,hybrid,None,None,None
654,Diesel Duff,hybrid,3.8,"Euphoric,Happy,Talkative,Giggly,Uplifted","Sweet,Flowery,Honey"


In [160]:
df1 = df['effects'].str.get_dummies(sep=',')

In [161]:
df1 = pd.merge(df, df1, left_index=True, right_index=True)
df1.head()

,strain,type,rating,effects,taste,Aroused,Creative,Dry,Energetic,Euphoric,Focused,Giggly,Happy,Hungry,Mouth,None,Relaxed,Sleepy,Talkative,Tingly,Uplifted
0,100 Og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0
1,98 White Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1
3,13 Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1
4,24K Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange",0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1


In [162]:
tastes = df['taste'].str.get_dummies(sep=',')

In [163]:
df1 = df1.drop(columns=['effects','rating','taste'])

In [164]:
df2 = df
df2.head()

,strain,type,rating,effects,taste
0,100 Og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus"
1,98 White Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel"
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody"
3,13 Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit"
4,24K Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange"


In [154]:
df2 = pd.merge(df2, tastes, left_index=True, right_index=True)
df2.head()

,strain,type,rating,effects,taste,Ammonia,Apple,Apricot,Berry,Blue,Blueberry,Butter,Cheese,Chemical,Chestnut,Citrus,Coffee,Diesel,Earthy,Flowery,Fruit,Grape,Grapefruit,Honey,Lavender,Lemon,Lime,Mango,Menthol,Mint,Minty,None,Nutty,Orange,Peach,Pear,Pepper,Pine,Pineapple,Plum,Pungent,Rose,Sage,Skunk,Spicy/Herbal,Strawberry,Sweet,Tar,Tea,Tobacco,Tree,Tropical,Vanilla,Violet,Woody
0,100 Og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,98 White Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
3,13 Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,24K Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange",0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [156]:
df2 = df2.drop(columns=['effects','rating','taste'])
df2.head()

,strain,type,Ammonia,Apple,Apricot,Berry,Blue,Blueberry,Butter,Cheese,Chemical,Chestnut,Citrus,Coffee,Diesel,Earthy,Flowery,Fruit,Grape,Grapefruit,Honey,Lavender,Lemon,Lime,Mango,Menthol,Mint,Minty,None,Nutty,Orange,Peach,Pear,Pepper,Pine,Pineapple,Plum,Pungent,Rose,Sage,Skunk,Spicy/Herbal,Strawberry,Sweet,Tar,Tea,Tobacco,Tree,Tropical,Vanilla,Violet,Woody
0,100 Og,hybrid,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,98 White Widow,hybrid,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1024,sativa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
3,13 Dawgs,hybrid,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,24K Gold,hybrid,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [165]:
df1.head()

,strain,type,Aroused,Creative,Dry,Energetic,Euphoric,Focused,Giggly,Happy,Hungry,Mouth,None,Relaxed,Sleepy,Talkative,Tingly,Uplifted
0,100 Og,hybrid,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0
1,98 White Widow,hybrid,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0
2,1024,sativa,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1
3,13 Dawgs,hybrid,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1
4,24K Gold,hybrid,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1


In [183]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set_style('darkgrid')
fig = plt.figure(figsize = (20,16))
fig.subplots_adjust(hspace = .30)

ax1 = fig.add_subplot(321)
ax1.hist(df1['type'], bins = 20, alpha = .50)
ax1.set_xlabel('Pclass', fontsize = 15)
ax1.set_ylabel('# Passengers',fontsize = 15)
ax1.set_title('Passenger Class',fontsize = 15)

# ax2 = fig.add_subplot(323)
# ax2.hist(titanic['Age'], bins = 20, alpha = .50,edgecolor= 'black',color ='teal')
# ax2.set_xlabel('Age',fontsize = 15)
# ax2.set_ylabel('# Passengers',fontsize = 15)
# ax2.set_title('Age of Passengers',fontsize = 15)

Text(0.5, 1.0, 'Passenger Class')

### Connect to The Strain API Data

In [179]:
import requests
response = requests.get('http://strainapi.evanbusse.com/oJPpcAq/strains/search/all')
data = response.json()
df3 = pd.DataFrame.from_dict(data)
df3 = df.transpose()
# df3 = pd.concat([df3.drop(['effects'], axis=1), df3['effects'].apply(pd.Series)], axis=1)
df3.reset_index(inplace=True)
df3.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,...,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147,2148,2149,2150,2151,2152,2153,2154,2155,2156,2157,2158,2159,2160,2161,2162,2163,2164,2165,2166,2167,2168,2169,2170,2171,2172,2173,2174,2175,2176,2177,2178,2179,2180,2181,2182,2183,2184,2185,2186,2187,2188,2189,2190,2191,2192,2193,2194,2195,2196,2197,2198,2199,2200,2201,2202,2203,2204,2205,2206,2207,2208,2209,2210,2211,2212,2213,2214,2215,2216,2217,2218,2219,2220,2221,2222,2223,2224,2225,2226,2227,2228,2229,2230,2231,2232,2233,2234,2235,2236,2237,2238,2239,2240,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250,2251,2252,2253,2254,2255,2256,2257,2258,2259,2260,2261,2262,2263,2264,2265,2266,2267,2268,2269,2270,2271,2272,2273,2274
0,strain,100 Og,98 White Widow,1024,13 Dawgs,24K Gold,3 Bears Og,3 Kings,303 Og,3D Cbd,3X Crazy,3Rd Coast Panama Chunk,501St Og,5Th Element,707 Headband,8 Ball Kush,818 Og,831 Og,9 Pound Hammer,91 Krypt,A 10,A Dub,A Train,Acdc,Ak 47,Ak 48,Aberdeen,Abusive Og,Acapulco Gold,Ace Killer Og,Ace Of Spades,Aceh,Acid,Acid Dough,Afcrack,Afghan Big Bud,Afghan Cow,Afghan Diesel,Afghan Hawaiian,Afghan Haze,Afghan Kush,Afghan Skunk,Afghan Sour Kush,Afghan Widow,Afghani,Afgahni Bullrider,Afghani Cbd,Afghanica,Afghanimal,Afghooey,Afgoo,Afgooey,Afkansastan,Afpak,African,Afrikaner,Afternoon Delight,Afwreck,Agent Orange,Agent Tangie,Air Jordan Og,Als Dream,Alaska,Alaska Thunder Grape,Alaskan Ice,Alaskan Thunder Fuck,Albert Walker,Alchemy,Alf,Alice In Wonderland,Alien Abduction,Alien Apparition,Alien Asshat,Alien Bubba,Alien Dawg,Alien Dutchess,Alien Hallucination,Alien Inferno,Alien Kush,Alien Og,Alien Reunion,Alien Rift,Alien Rock Candy,Alien Sour Apple,Alien Stardawg,Alien Technology,Aliens On Moonshine,Allen Iverson Og,Allen Wrench,Allkush,Aloha,Aloha Grape Stomper,Aloha Limone,Alohaberry,Alpha Blue,Alpha Cow,Alpha Express,Alpha Og,Alpine Blue,Alpine Star,Ambrosia,American Dream,American Kush,Americano,Amethyst Bud,Amnesia,Amnesia Ganja Haze,Amnesia Haze,Ancient Kush,Ancient Og,Anesthesia,Angel Og,Animal Cookies,Anonymous Og,Anubis,Ape Shit,Apollo 11,Apollo 13,Appalachia,Appalachian Power,Apple Jack,Apple Kush,Appleberry,Arctic Sun,Arcticblue,Area 51,Argyle,Arjans Strawberry Haze,Arjans Ultra Haze 1,Armageddon,Armagnac,Ash,Asian Fantasy,Aspen Og,Astroboy,Athabasca,Atmosphere,Atomic,Atomic Goat,Atomic Northern Lights,Atomical Haze,Aurora Borealis,Aurora Indica,Avalon,Avi,Avi Dekel,Ayahuasca Purple,Azure Haze,B 52,B Witched,B Witched,B Witched,B Witched,Bb King,B4,Bc Big Bud,Bc Roadkill,Bc Sweet Tooth,Bcn Diesel,Bog Bubble,Bsc,Bakerstreet

In [177]:
df4.tail()

,index,strain,type,rating,taste,0
2270,2270,Zeus Og,hybrid,4.7,"Earthy,Woody,Pine","Happy,Uplifted,Relaxed,Euphoric,Energetic"
2271,2271,Zkittlez,indica,4.6,"Sweet,Berry,Grape","Relaxed,Happy,Euphoric,Uplifted,Sleepy"
2272,2272,Zombie Kush,indica,5,"Earthy,Sweet,Spicy/Herbal","Relaxed,Sleepy,Talkative,Euphoric,Happy"
2273,2273,Zombie Og,indica,4.4,"Sweet,Earthy,Pungent","Relaxed,Sleepy,Euphoric,Happy,Hungry"
2274,2274,Zoom Pie,indica,4.6,"Berry,Earthy,Pungent","Hungry,Relaxed,Uplifted,Happy,Sleepy"


In [172]:
# df3.drop(columns=['index'], inplace=True)
df3 = df.rename(columns={'strain':'name'})
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2275 entries, 0 to 2274
Data columns (total 5 columns):
name       2275 non-null object
type       2275 non-null object
rating     2275 non-null object
effects    2275 non-null object
taste      2275 non-null object
dtypes: object(5)
memory usage: 106.6+ KB


In [174]:
df3.head()

,name,type,rating,effects,taste
0,100 Og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus"
1,98 White Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel"
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody"
3,13 Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit"
4,24K Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange"


In [173]:
# List for dicts for easy dataframe creation
dict_list = []
# iterating over df with more strains
for name in df1.strain:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, df1.strain, 90)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"strain" : name})
    dict_.update({"name" : match[0]})
    dict_list.append(dict_)
    
merge_table1 = pd.DataFrame(dict_list)
# Display results
merge_table1

,name,strain
0,100 Og,100 Og
1,98 White Widow,98 White Widow
2,1024,1024
3,13 Dawgs,13 Dawgs
4,24K Gold,24K Gold
5,3 Bears Og,3 Bears Og
6,3 Kings,3 Kings
7,303 Og,303 Og
8,3D Cbd,3D Cbd
9,3X Crazy,3X Crazy
